In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from cipher import Ring, Parameters, Context 
from scheme import Evaluator, Encryptor, Decryptor, Encoder

## Test configuration

In [3]:
abs_tol = 1e-5 # error tolerance

## Context set up

In [4]:
myring = Ring(seed=1234)
parms = Parameters(logp = 30, logq = 150, logn = 12)
context = Context(parms, myring)

sk = context.generate_secret_key()

print(context.params)

 logp: 30
 logq: 150 
 logn: 12


## Create Agents

In [5]:
keys = {"mult":context.generate_mult_key(),
        "rot":{'1':'hi1',
               '2':'hi2',
               '4':'hi4',
               '8':'hi8'}}

ev = Evaluator(keys, context)
encryptor = Encryptor(context)

decryptor = Decryptor(sk)

encoder = Encoder(context)

## Create a ctxt and attempt to print it 
Both ndarray and list are OK

In [6]:
ctxt1 = encryptor.encrypt(np.array([1,2,3,4]))
ctxt2 = encryptor.encrypt([2,3,3,5])
print(ctxt1)

You can't read the content


#### You need to first decrypt it

In [7]:
decrypted = decryptor.decrypt(ctxt2)
print(decrypted)

[2. 3. 3. ... 0. 0. 0.]


## Addition

In [8]:
new_ctxt = ev.add(ctxt1, ctxt2, inplace=False)

print(decryptor.decrypt(new_ctxt))

[3. 5. 6. ... 0. 0. 0.]


### Mult

In [9]:
ctxt3 = ev.mult(ctxt1, ctxt2, inplace=False)

print(decryptor.decrypt(ctxt3))

ev.mult(ctxt1, ctxt2, inplace=True)
print(decryptor.decrypt(ctxt1))

[2. 6. 9. ... 0. 0. 0.]
[2. 6. 9. ... 0. 0. 0.]


### mult by plain

ptxt can be a
1. scalar
2. single-element sequence 
3. n_element-long sequence
4. nslots-long sequence

In [10]:
ptxt = encoder.encode([5])
ev.mult_by_plain(ctxt1, ptxt, inplace=True)

ScaleMisMatchError: Scales of Ctxts don't match

In [11]:
context.logq

150

In [12]:
ev.rescale_next(ctxt1) # match scale
ev.mult_by_plain(ctxt1, ptxt, inplace=True)
print(decryptor.decrypt(ctxt1))

[10.  0.  0. ...  0.  0.  0.]


In [13]:
context.logq

120

## Algorithms
### sum

In [11]:
from algorithms import Algorithms 
algo = Algorithms(ev)

summed = algo.sum_reduce(ctxt1, partial=False)

_dec = decryptor.decrypt(summed) # if partial=True, only the first slot if valid.
assert np.isclose(np.sum(ctxt1._arr), _dec[0], atol=abs_tol), "Error!"